In [ ]:
!git clone https://github.com/geekyutao/Inpaint-Anything

Cloning into 'Inpaint-Anything'...
remote: Enumerating objects: 2006, done.
remote: Counting objects: 100% (355/355), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 2006 (delta 100), reused 315 (delta 87), pack-reused 1651
Receiving objects: 100% (2006/2006), 294.43 MiB | 21.65 MiB/s, done.
Resolving deltas: 100% (571/571), done.
Updating files: 100% (675/675), done.


In [ ]:
#mount the github repo folder 
import sys
sys.path.insert(0,'/content/Inpaint-Anything')
sys.path.insert(0,'/content/Inpaint-Anything/segment_anything')

In [ ]:
!pip install torch torchvision torchaudio
!pip install segment_anything
!pip install -r ./Inpaint-Anything/lama/requirements.txt 
!pip install diffusers transformers accelerate scipy safetensors
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 101.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.9/841.9 kB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 33.6 MB/s eta 0:00:00
 

In [ ]:
!gdown 1KAzTikaKmMyuC_dhRAuav64pH7j9OVjo
!unzip pretrained_models.zip
!mv big-lama Inpaint-Anything/pretrained_models/
!mv sam_vit_h_4b8939.pth Inpaint-Anything/pretrained_models/
!mv sttn.pth Inpaint-Anything/pretrained_models/
!rm -rf pretrained_models.zip

Downloading...
From: https://drive.google.com/uc?id=1KAzTikaKmMyuC_dhRAuav64pH7j9OVjo
To: /content/pretrained_models.zip
100% 2.82G/2.82G [00:39<00:00, 71.9MB/s]
Archive:  pretrained_models.zip
  inflating: big-lama/.DS_Store      
  inflating: big-lama/config.yaml    
  inflating: big-lama/models/best.ckpt  
  inflating: sam_vit_h_4b8939.pth    
  inflating: sttn.pth                


In [ ]:
import cv2
import sys
import argparse
import numpy as np
import torch
from pathlib import Path
from matplotlib import pyplot as plt
from typing import Any, Dict, List
from sam_segment import predict_masks_with_sam
from stable_diffusion_inpaint import fill_img_with_sd
from utils import load_img_to_array, save_array_to_img, dilate_mask, show_mask, show_points, get_clicked_point
import gradio as gr
import os
import tempfile
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry


In [ ]:
def setup_args(parser):
    # parser.add_argument(
    #     "--input_img", type=str, required=True,
    #     help="Path to a single input img",
    # )
    # parser.add_argument(
    #     "--coords_type", type=str, required=True,
    #     default="key_in", choices=["click", "key_in"], 
    #     help="The way to select coords",
    # )
    # parser.add_argument(
    #     "--point_coords", type=float, nargs='+', required=True,
    #     help="The coordinate of the point prompt, [coord_W coord_H].",
    # )
    # parser.add_argument(
    #     "--point_labels", type=int, nargs='+', required=True,
    #     help="The labels of the point prompt, 1 or 0.",
    # )
    # parser.add_argument(
    #     "--text_prompt", type=str, required=True,
    #     help="Text prompt",
    # )
    # parser.add_argument(
    #     "--dilate_kernel_size", type=int, default=None,
    #     help="Dilate kernel size. Default: None",
    # )
    # parser.add_argument(
    #     "--output_dir", type=str, required=True,
    #     help="Output path to the directory with results.",
    # )
    parser.add_argument(
        "--sam_model_type", type=str,
        default="vit_h", choices=['vit_h', 'vit_l', 'vit_b'],
        help="The type of sam model to load. Default: 'vit_h"
    )
    parser.add_argument(
        "--sam_ckpt", type=str, 
        default="/content/Inpaint-Anything/pretrained_models/sam_vit_h_4b8939.pth",
        help="The path to the SAM checkpoint to use for mask generation.",
    )
    parser.add_argument(
        "--seed", type=int,
        help="Specify seed for reproducibility.",
    )
    parser.add_argument(
        "--deterministic", action="store_true",
        help="Use deterministic algorithms for reproducibility.",
    )

In [ ]:
def mkstemp(suffix, dir=None):
    fd, path = tempfile.mkstemp(suffix=f"{suffix}", dir=dir)
    os.close(fd)
    return Path(path)

In [ ]:
def get_sam_feat(img):
    model['sam'].set_image(img)
    features = model['sam'].features 
    orig_h = model['sam'].orig_h 
    orig_w = model['sam'].orig_w 
    input_h = model['sam'].input_h 
    input_w = model['sam'].input_w 
    model['sam'].reset_image()
    return features, orig_h, orig_w, input_h, input_w

In [ ]:
def get_masked_img(img, w, h, features, orig_h, orig_w, input_h, input_w, dilate_kernel_size):
    point_coords = [w, h]
    point_labels = [1]

    model['sam'].is_image_set = True
    model['sam'].features = features
    model['sam'].orig_h = orig_h
    model['sam'].orig_w = orig_w
    model['sam'].input_h = input_h
    model['sam'].input_w = input_w

    # model['sam'].set_image(img) # todo : update here for accelerating
    masks, _, _ = model['sam'].predict(
        point_coords=np.array([point_coords]),
        point_labels=np.array(point_labels),
        multimask_output=True,
    )

    masks = masks.astype(np.uint8) * 255

    # dilate mask to avoid unmasked edge effect
    if dilate_kernel_size is not None:
        masks = [dilate_mask(mask, dilate_kernel_size) for mask in masks]
    else:
        masks = [mask for mask in masks]

    figs = []
    for idx, mask in enumerate(masks):
        # save the pointed and masked image
        tmp_p = mkstemp(".png")
        dpi = plt.rcParams['figure.dpi']
        height, width = img.shape[:2]
        fig = plt.figure(figsize=(width/dpi/0.77, height/dpi/0.77))
        plt.imshow(img)
        plt.axis('off')
        show_points(plt.gca(), [point_coords], point_labels,
                    size=(width*0.04)**2)
        show_mask(plt.gca(), mask, random_color=False)
        plt.tight_layout()
        plt.savefig(tmp_p, bbox_inches='tight', pad_inches=0)
        figs.append(fig)
        plt.close()
    return *figs, *masks

In [ ]:
def get_filled_img(img, mask0, mask1, mask2, text_prompt):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  out = []
  for mask in [mask0, mask1, mask2]:
    if len(mask.shape)==3:
            mask = mask[:,:,0]
    img_filled = fill_img_with_sd(img, mask, text_prompt, device=device)
    img_filled = img_filled.astype(np.uint8)
    out.append(img_filled)
  return out

In [ ]:
# Building models
# get args 

parser = argparse.ArgumentParser()
setup_args(parser)
#args = parser.parse_args(sys.argv[1:])
#for notebook only this works
args = parser.parse_args("")

# build models
model = {}
# build the sam model
model_type="vit_h"
ckpt_p=args.sam_ckpt
model_sam = sam_model_registry[model_type](checkpoint=ckpt_p)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_sam.to(device=device)
model['sam'] = SamPredictor(model_sam)

In [ ]:
#building the ui
button_size = (100,50)
with gr.Blocks() as demo:
    features = gr.State(None)
    orig_h = gr.State(None)
    orig_w = gr.State(None)
    input_h = gr.State(None)
    input_w = gr.State(None)

    with gr.Row().style(mobile_collapse=False, equal_height=True):
        with gr.Column(variant="panel"):
            with gr.Row():
                gr.Markdown("## Input Image")
            with gr.Row():
                img = gr.Image(label="Input Image").style(height="200px")
        with gr.Column(variant="panel"):
            with gr.Row():
                gr.Markdown("## Pointed Image")
            with gr.Row():
                img_pointed = gr.Plot(label='Pointed Image')
        with gr.Column(variant="panel"):
            with gr.Row():
                gr.Markdown("## Control Panel")
            with gr.Row():
                w = gr.Number(label="Point Coordinate W")
                h = gr.Number(label="Point Coordinate H")
            dilate_kernel_size = gr.Slider(label="Dilate Kernel Size", minimum=0, maximum=100, step=1, value=15)
            sam_mask = gr.Button("Predict Mask", variant="primary").style(full_width=True, size="sm")
            with gr.Row():
              text_prompt = gr.Textbox(label="Enter text prompt")
              fillAny = gr.Button("Fill", variant="primary").style(full_width=True, size="sm")
            # lama = gr.Button("Inpaint Image", variant="primary").style(full_width=True, size="sm")
            clear_button_image = gr.Button(value="Reset", label="Reset", variant="secondary").style(full_width=True, size="sm")

    # todo: maybe we can delete this row, for it's unnecessary to show the original mask for customers
    with gr.Row(variant="panel"):
        with gr.Column():
            with gr.Row():
                gr.Markdown("## Segmentation Mask")
            with gr.Row():
                mask_0 = gr.outputs.Image(type="numpy", label="Segmentation Mask 0").style(height="200px")
                mask_1 = gr.outputs.Image(type="numpy", label="Segmentation Mask 1").style(height="200px")
                mask_2 = gr.outputs.Image(type="numpy", label="Segmentation Mask 2").style(height="200px")

    with gr.Row(variant="panel"):
        with gr.Column():
            with gr.Row():
                gr.Markdown("## Image with Mask")
            with gr.Row():
                img_with_mask_0 = gr.Plot(label="Image with Segmentation Mask 0")
                img_with_mask_1 = gr.Plot(label="Image with Segmentation Mask 1")
                img_with_mask_2 = gr.Plot(label="Image with Segmentation Mask 2")

    with gr.Row(variant="panel"):
        with gr.Column():
            with gr.Row():
                gr.Markdown("## Image Filled with Mask ")
            with gr.Row():
                img_rm_with_mask_0 = gr.outputs.Image(
                    type="numpy", label="Image Filled with Segmentation Mask 0").style(height="200px")
                img_rm_with_mask_1 = gr.outputs.Image(
                    type="numpy", label="Image Filled with Segmentation Mask 1").style(height="200px")
                img_rm_with_mask_2 = gr.outputs.Image(
                    type="numpy", label="Image Filled with Segmentation Mask 2").style(height="200px")


    def get_select_coords(img, evt: gr.SelectData):
        dpi = plt.rcParams['figure.dpi']
        height, width = img.shape[:2]
        fig = plt.figure(figsize=(width/dpi/0.77, height/dpi/0.77))
        plt.imshow(img)
        plt.axis('off')
        plt.tight_layout()
        show_points(plt.gca(), [[evt.index[0], evt.index[1]]], [1],
                    size=(width*0.04)**2)
        return evt.index[0], evt.index[1], fig

    img.select(get_select_coords, [img], [w, h, img_pointed])
    img.upload(get_sam_feat, [img], [features, orig_h, orig_w, input_h, input_w])

    sam_mask.click(
        get_masked_img,
        [img, w, h, features, orig_h, orig_w, input_h, input_w, dilate_kernel_size],
        [img_with_mask_0, img_with_mask_1, img_with_mask_2, mask_0, mask_1, mask_2]
    )

    fillAny.click(
        get_filled_img,
        [img, mask_0, mask_1, mask_2, text_prompt],
        [img_rm_with_mask_0, img_rm_with_mask_1, img_rm_with_mask_2]
    )


    def reset(*args):
        return [None for _ in args]

    clear_button_image.click(
        reset,
        [img, features, img_pointed, w, h, mask_0, mask_1, mask_2, img_with_mask_0, img_with_mask_1, img_with_mask_2, img_rm_with_mask_0, img_rm_with_mask_1, img_rm_with_mask_2],
        [img, features, img_pointed, w, h, mask_0, mask_1, mask_2, img_with_mask_0, img_with_mask_1, img_with_mask_2, img_rm_with_mask_0, img_rm_with_mask_1, img_rm_with_mask_2]
    )

/usr/local/lib/python3.10/dist-packages/gradio/layouts.py:73: UserWarning: mobile_collapse is no longer supported.
  warnings.warn("mobile_collapse is no longer supported.")
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


In [ ]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>